# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)


In [1]:
import warnings

warnings.filterwarnings(action='ignore')
import paddle
from paddle import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = paddle.rand([2, 4])
net(X)

Tensor(shape=[2, 1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[0.48685262],
        [0.34589919]])

## [**参数访问**]

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。


In [2]:
print(net[2].state_dict())

OrderedDict([('weight', Parameter containing:
Tensor(shape=[8, 1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[ 0.35208166],
        [-0.81014472],
        [ 0.77976549],
        [-0.21038508],
        [ 0.06733459],
        [-0.73830664],
        [ 0.65213728],
        [-0.33782813]])), ('bias', Parameter containing:
Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.]))])


输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### [**目标参数**]

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.value)

<class 'paddle.fluid.framework.ParamBase'>
Parameter containing:
Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.])
<bound method PyCapsule.value of Parameter containing:
Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.])>


参数是复合的对象，包含值、梯度和额外信息。
这就是我们需要显式参数值的原因。
除了值之外，我们还可以访问每个参数的梯度。
在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。


In [4]:
net[2].weight.grad == None

True

### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', [4, 8]) ('bias', [8])
('0.weight', [4, 8]) ('0.bias', [8]) ('2.weight', [8, 1]) ('2.bias', [1])


这为我们提供了另一种访问网络参数的方式，如下所示。


In [6]:
net.state_dict()['2.bias']

Parameter containing:
Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.])

### [**从嵌套块收集参数**]

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_sublayer(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

Tensor(shape=[2, 1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[0.],
        [0.]])

[**设计了网络后，我们看看它是如何工作的。**]


In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, dtype=float32)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, dtype=float32)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, dtype=float32)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, dtype=float32)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, dtype=float32)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, dtype=float32)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, dtype=float32)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, dtype=float32)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, dtype=float32)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。


In [9]:
print(rgnet[0].state_dict()['block 0.0.bias'])

Parameter containing:
Tensor(shape=[8], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0., 0., 0., 0., 0., 0., 0., 0.])


## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在 :numref:`sec_numerical_stability`中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。


默认情况下，PaddlePaddle会使用Xavier初始化权重矩阵，
偏置参数设置为0。
PaddlePaddle的`nn.initializer`模块提供了多种预置初始化方法。


### [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        paddle.nn.initializer.Normal(mean=0.0, std=0.01)
        paddle.zeros(m.bias)
net.apply(init_normal)
net[0].weight[0],net[0].state_dict()['bias']

(Tensor(shape=[8], dtype=float32, place=Place(cpu), stop_gradient=False,
        [-0.45191017,  0.32920808, -0.20261598,  0.45921832,  0.20553625,
         -0.44192296,  0.23093420,  0.31619781]),
 Parameter containing:
 Tensor(shape=[8], dtype=float32, place=Place(cpu), stop_gradient=False,
        [0., 0., 0., 0., 0., 0., 0., 0.]))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。


In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        paddle.nn.initializer.Constant(value = 1)
        paddle.zeros(m.bias)
net.apply(init_constant)
net[0].weight[0],net[0].state_dict()['bias']

(Tensor(shape=[8], dtype=float32, place=Place(cpu), stop_gradient=False,
        [-0.45191017,  0.32920808, -0.20261598,  0.45921832,  0.20553625,
         -0.44192296,  0.23093420,  0.31619781]),
 Parameter containing:
 Tensor(shape=[8], dtype=float32, place=Place(cpu), stop_gradient=False,
        [0., 0., 0., 0., 0., 0., 0., 0.]))

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
然后将第三个神经网络层初始化为常量值42。


In [12]:
def xavier(m):
    if type(m) == nn.Linear:
        paddle.nn.initializer.XavierUniform(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        paddle.nn.initializer.Constant(42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight[0])
print(net[2].weight)

Tensor(shape=[8], dtype=float32, place=Place(cpu), stop_gradient=False,
       [-0.45191017,  0.32920808, -0.20261598,  0.45921832,  0.20553625,
        -0.44192296,  0.23093420,  0.31619781])
Parameter containing:
Tensor(shape=[8, 1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[ 0.35208166],
        [-0.81014472],
        [ 0.77976549],
        [-0.21038508],
        [ 0.06733459],
        [-0.73830664],
        [ 0.65213728],
        [-0.33782813]])


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


同样，我们实现了一个`my_init`函数来应用到`net`。


In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        paddle.nn.initializer.XavierUniform(m.weight, -10, 10)
        h = paddle.abs(m.weight) >= 5
        h = paddle.to_tensor(h)
        m = paddle.to_tensor(m.weight)
        m *= h

net.apply(my_init)
net[0].weight[:2]

Init weight [4, 8]
Init weight [8, 1]


Tensor(shape=[2, 8], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[-0.45191017,  0.32920808, -0.20261598,  0.45921832,  0.20553625,
         -0.44192296,  0.23093420,  0.31619781],
        [-0.34684724, -0.05426282, -0.12417591, -0.11773884,  0.51981491,
          0.52964920,  0.54408771, -0.02475590]])

注意，我们始终可以直接设置参数。


In [14]:
net[0].weight.set_value(net[0].weight.numpy() + 1)
val = net[0].weight.numpy()
val[0, 0] = 42
net[0].weight.set_value(val)
net[0].weight[0]

Tensor(shape=[8], dtype=float32, place=Place(cpu), stop_gradient=False,
       [42.       , 1.32920814, 0.79738402, 1.45921826, 1.20553625, 0.55807704,
        1.23093414, 1.31619787])

## [**参数绑定**]

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [15]:
# 我们需要给共享层一个名称，以便可以引用它的参数。
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight[0] == net[4].weight[0])

Tensor(shape=[8], dtype=bool, place=Place(cpu), stop_gradient=False,
       [True, True, True, True, True, True, True, True])


## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用 :numref:`sec_model_construction` 中定义的`FancyMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？


[Discussions](https://discuss.d2l.ai/t/11778)
